In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torchvision import transforms

/usr/lib/python3.6/site-packages/requests/__init__.py:80: RequestsDependencyWarning: urllib3 (1.23) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [2]:
transform = transforms.Compose([transforms.ToTensor()])

In [3]:
trainset = torchvision.datasets.FashionMNIST(root="data/fashion-mnist", train=True, transform=transform, download=False)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4, shuffle=True, num_workers=2)

In [4]:
testset = torchvision.datasets.FashionMNIST(root="data/fashion-mnist", train=False, transform=transform, download=False)
testloader = torch.utils.data.DataLoader(testset, batch_size=1, num_workers=2)

In [5]:
classes = (0, 1, 2, 3, 4, 5, 6, 7, 8, 9)

In [6]:
class DDMLNet(nn.Module):
    def __init__(self):
        super(DDMLNet, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5, padding=2), 
            nn.ReLU(), 
            nn.MaxPool2d(2, 2)
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5, padding=2), 
            nn.ReLU(), 
            nn.MaxPool2d(2, 2)
        )
        self.fc1 = nn.Linear(16 * 7 * 7, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = x.view(-1, 16 * 7 * 7)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [7]:
net = DDMLNet()

In [8]:
import torch.optim as optim

In [9]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [10]:
for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 100 == 0:
            print('[%d, %5d] loss: %.3f' % (epoch, i, running_loss / 100))
            running_loss = 0.0

print('Finished Training')

[0,     0] loss: 0.023
[0,   100] loss: 2.299
[0,   200] loss: 2.287
[0,   300] loss: 2.275
[0,   400] loss: 2.206
[0,   500] loss: 1.841
[0,   600] loss: 1.403
[0,   700] loss: 1.262
[0,   800] loss: 1.176
[0,   900] loss: 1.105
[0,  1000] loss: 1.022
[0,  1100] loss: 1.095
[0,  1200] loss: 1.054
[0,  1300] loss: 1.000
[0,  1400] loss: 0.884
[0,  1500] loss: 1.006
[0,  1600] loss: 0.774
[0,  1700] loss: 0.812
[0,  1800] loss: 0.783
[0,  1900] loss: 0.901
[0,  2000] loss: 0.734
[0,  2100] loss: 0.803
[0,  2200] loss: 0.733
[0,  2300] loss: 0.848
[0,  2400] loss: 0.818
[0,  2500] loss: 0.749
[0,  2600] loss: 0.886
[0,  2700] loss: 0.711
[0,  2800] loss: 0.698
[0,  2900] loss: 0.734
[0,  3000] loss: 0.642
[0,  3100] loss: 0.758
[0,  3200] loss: 0.695
[0,  3300] loss: 0.646
[0,  3400] loss: 0.691
[0,  3500] loss: 0.704
[0,  3600] loss: 0.694
[0,  3700] loss: 0.660
[0,  3800] loss: 0.695
[0,  3900] loss: 0.668
[0,  4000] loss: 0.595
[0,  4100] loss: 0.678
[0,  4200] loss: 0.595
[0,  4300] 

In [29]:
currcet = 0

for inputs, labels in testloader:
    outputs = net(inputs)
    value, result = torch.max(outputs, dim=1)

    if result == labels:
        currcet += 1

In [31]:
currcet/len(testloader)

0.8744

In [11]:
p = list(net.parameters())

In [57]:
id(p[5].data)

140061776948320

In [68]:
p[5].data

tensor([-1.2503,  0.6235,  0.3977, -0.9062,  3.1390, -0.3585, -0.3160,
         1.2803, -1.9810, -0.2359, -0.5886, -0.3396,  0.1739, -0.4158,
        -0.3605, -1.3266, -0.7727, -1.4952,  0.4273,  0.3919, -1.7949,
         0.5489, -2.8924, -0.2457, -0.6428, -1.2991, -0.7892,  0.7772,
         0.8509,  0.4290,  0.5687,  1.2908, -0.7518,  0.6406, -0.4864,
         1.3753,  1.7053,  1.5062, -2.1529, -0.2778, -1.4133, -1.0603,
        -1.0310,  0.6597, -0.8367, -1.1041, -0.5733,  1.0010,  1.1177,
        -1.0557, -1.1679,  0.0086, -0.9557, -0.5922,  0.6309,  1.7029,
         0.2819,  0.8734, -0.4209,  1.0276, -1.1666,  0.3717, -1.3816,
         0.7741, -0.4853,  2.1810, -0.0510,  0.3209, -0.1571,  0.5306,
         1.2918,  1.0591, -1.3354,  0.4622, -0.8600, -0.8014, -1.0855,
         0.2888,  0.0252, -0.4255,  0.8408, -0.2051,  1.4430,  0.4288])

In [59]:
p_fc1 = list(net.fc1.parameters())

In [62]:
id(p_fc1[1].data)

140061721570904

In [67]:
p_fc1[1].data = torch.randn(84)
p_fc1[1].data

tensor([-1.2503,  0.6235,  0.3977, -0.9062,  3.1390, -0.3585, -0.3160,
         1.2803, -1.9810, -0.2359, -0.5886, -0.3396,  0.1739, -0.4158,
        -0.3605, -1.3266, -0.7727, -1.4952,  0.4273,  0.3919, -1.7949,
         0.5489, -2.8924, -0.2457, -0.6428, -1.2991, -0.7892,  0.7772,
         0.8509,  0.4290,  0.5687,  1.2908, -0.7518,  0.6406, -0.4864,
         1.3753,  1.7053,  1.5062, -2.1529, -0.2778, -1.4133, -1.0603,
        -1.0310,  0.6597, -0.8367, -1.1041, -0.5733,  1.0010,  1.1177,
        -1.0557, -1.1679,  0.0086, -0.9557, -0.5922,  0.6309,  1.7029,
         0.2819,  0.8734, -0.4209,  1.0276, -1.1666,  0.3717, -1.3816,
         0.7741, -0.4853,  2.1810, -0.0510,  0.3209, -0.1571,  0.5306,
         1.2918,  1.0591, -1.3354,  0.4622, -0.8600, -0.8014, -1.0855,
         0.2888,  0.0252, -0.4255,  0.8408, -0.2051,  1.4430,  0.4288])